In [1]:
import pandas as pd
import numpy as np
import random
from ortools.sat.python import cp_model

In [2]:
aulas = pd.read_excel('/home/cayena/Downloads/planilha_teste_thefamily.xlsx')

In [3]:
aulas['Horário'] = aulas['Horário'].astype(str)

In [4]:
aulas['segunda'] = aulas['Dias da Semana'].str.contains('2ª').astype(int)
aulas['terça'] = aulas['Dias da Semana'].str.contains('3ª').astype(int)
aulas['quarta'] = aulas['Dias da Semana'].str.contains('4ª').astype(int)
aulas['quinta'] = aulas['Dias da Semana'].str.contains('5ª').astype(int)

In [5]:
df_horarios = aulas['Horário'].str.get_dummies()
df_unidade = aulas['Unidade'].str.get_dummies()

In [6]:
aulas_geral = pd.concat([aulas, df_horarios,df_unidade], axis=1)

In [7]:
grupo_semana = aulas_geral[['Grupo', 'segunda', 'terça', 'quarta', 'quinta']].copy()
grupo_horario = aulas_geral.drop(columns=[ 'Horário', 'Unidade', 'Dias da Semana','segunda', 'terça', 'quarta', 'quinta', 'Jardim', 'Satélite', 'Vicentina']).copy()
grupo_loc = aulas_geral[['Grupo', 'Jardim','Satélite','Vicentina']].copy()

In [8]:
Professores = {'Name':['Allan','Carlos','Henrique','Yasmin','Malu','Tanze','Okamoto','Edu','Wolf','Odin','Renan','Gabiraes','Nicolas','Novaes','Jorge','Kaori']}
profes = pd.DataFrame(Professores)
profes.shape

(16, 1)

In [9]:
# sortear uma aula para cada professor
# usar a função de minimizar para completar os horarios e turmas restantes

In [9]:
aulas_segunda = aulas_geral[aulas_geral['segunda'] == 1].copy()

In [11]:
all_teacher = profes['Name'].to_list()
all_group = aulas_segunda['Grupo'].to_list()

In [11]:
all_teacher = profes['Name'].to_list()
all_group = aulas_segunda['Grupo'].to_list()

alocacoes = {}
model = cp_model.CpModel()
for i in all_teacher:
    for j in all_group:
        h = aulas_segunda.loc[aulas_segunda['Grupo'] == j, 'Horário'].values[0]
        alocacoes[(i,j,h)] = model.NewBoolVar(f"{i}_converinglesson_{j}_period_{h}")

for a in all_group:
    b = aulas_segunda.loc[aulas_segunda['Grupo'] == a, 'Horário'].values[0]
    model.Add(sum(alocacoes[(c,a,b)] for c in all_teacher) == 1)

for professor in all_teacher:
    for horario in aulas_segunda['Horário'].unique():
        model.Add(sum(alocacoes[(professor, row['Grupo'], horario)] for _, row in aulas_segunda.iterrows() if row['Horário'] == horario) <= 1)



In [17]:
model = cp_model.CpModel()

# Dicionário para armazenar as variáveis booleanas
alocacoes = {}

# Criar variáveis booleanas para as combinações de turma e horário existentes
for professor in all_teacher:
    for _, row in aulas_segunda.iterrows():
        turma = row['Grupo']
        horario = row['Horário']
        
        nome_var = f'{professor}_{turma}_{horario}'
        alocacoes[(professor, turma, horario)] = model.NewBoolVar(nome_var)

# Adicionar a restrição: um professor não pode estar em duas turmas no mesmo horário
for _, row in aulas_segunda.iterrows():
    turma = row['Grupo']
    horario = row['Horário']
    model.Add(sum(alocacoes[(professor, turma, horario)] for professor in all_teacher) == 1)

# Nova restrição: um professor só pode estar em uma turma por horário
for professor in all_teacher:
    for horario in aulas_segunda['Horário'].unique():
        model.Add(sum(alocacoes[(professor, row['Grupo'], horario)] for _, row in aulas_segunda.iterrows() if row['Horário'] == horario) <= 1)


# Criar o solver e rodar o modelo

solver = cp_model.CpSolver()
status = solver.Solve(model)
# Imprimir as alocações
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for professor in all_teacher:
        for _, row in aulas_segunda.iterrows():
            turma = row['Grupo']
            horario = row['Horário']# Nova restrição: um professor só pode estar em uma turma por horário
            # Verificar se o professor foi alocado para esta turma no horário específico
            if solver.BooleanValue(alocacoes[(professor, turma, horario)]):
                print(f"{professor} está alocado para {turma} no horário {horario}")
else:
    print("Nenhuma solução viável foi encontrada.")

Solução encontrada!
Allan está alocado para MBA IDEIAS EM AÇÃO no horário 19:00:00
Allan está alocado para SAVANAH PRESENCIAL no horário 07:00:00
Allan está alocado para FERNANDO CAJE - PRESENCIAL no horário 08:00:00
Allan está alocado para GARLAND PRESENCIAL no horário 18:00:00
Allan está alocado para FAIRFAX PRESENCIAL no horário 16:00:00
Allan está alocado para DELTA PRESENCIAL no horário 12:00:00
Allan está alocado para FALLS CHURCH PRESENCIAL no horário 20:30:00
Allan está alocado para DERBY PRESENCIAL no horário 09:00:00
Allan está alocado para SEATTLE PRESENCIAL no horário 15:00:00
Allan está alocado para LANCASTER PRESENCIAL no horário 17:00:00
Carlos está alocado para KNOXVILLE PRESENCIAL no horário 08:00:00
Carlos está alocado para GREAT BRITAIN PRESENCIAL no horário 09:00:00
Carlos está alocado para TAMPA PRESENCIAL no horário 19:00:00
Carlos está alocado para CAPE COD PRESENCIAL no horário 20:00:00
Carlos está alocado para WARRINGTON PRESENCIAL no horário 18:00:00
Carlos es